# Install

In [1]:
!pip install sentencepiece

     |████████████████████████████████| 1.2MB 5.5MB/s 


# Evn

In [2]:
import os
import random
import shutil
import json
import zipfile
import math
import copy
import collections
import re

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import sentencepiece as spm
import tensorflow as tf
import tensorflow.keras.backend as K

from tqdm.notebook import tqdm

In [3]:
# random seed initialize
random_seed = 1234
random.seed(random_seed)
np.random.seed(random_seed)
tf.random.set_seed(random_seed)

In [4]:
!nvidia-smi

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.



In [5]:
# google drive mount
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
# data dir
data_dir = '/content/drive/MyDrive/Colab Notebooks'
os.listdir(data_dir)

['Pandas 튜토리얼.ipynb',
 'Matplotlib 튜토리얼.ipynb의 사본',
 'Day1_EVN.ipynb의 사본',
 'Day1_Assignment.ipynb',
 'ko_32000.model',
 'ko_32000.vocab',
 'Day2_01_Encoding',
 'Day2_02_Tokenizer',
 'kowiki.txt.zip']

In [12]:
# kowiki dir
kowiki_dir = os.path.join(data_dir, 'kowiki')
if not os.path.exists(kowiki_dir):
    os.makedirs(kowiki_dir)
os.listdir(kowiki_dir)

['kowiki.txt.zip']

# Corpus

In [13]:
text = """위키백과의 최상위 도메인이 .com이던 시절 ko.wikipedia.com에 구판 미디어위키가 깔렸으나 한글 처리에 문제가 있어 글을 올릴 수도 없는 이름뿐인 곳이었다. 2002년 10월에 새로운 위키 소프트웨어를 쓰면서 한글 처리 문제가 풀리기 시작했지만, 가장 많은 사람이 쓰는 인터넷 익스플로러에서는 인코딩 문제가 여전했다. 이런 이유로 초기에는 도움말을 옮기거나 쓰는 일에 어려움을 겪었다. 이런 어려움이 있었는데도 위키백과 통계로는, 2002년 10월에서 2003년 7월까지 열 달 사이에 글이 13개에서 159개로 늘었고 2003년 7월과 8월 사이에는 한 달 만에 159개에서 348개로 늘어났다. 2003년 9월부터는 인터넷 익스플로러의 인코딩 문제가 사라졌으며, 대한민국 언론에서도 몇 차례 위키백과를 소개하면서 참여자가 점증하리라고 예측했다. 참고로 한국어 위키백과의 최초 문서는 2002년 10월 12일에 등재된 지미 카터 문서이다.
2005년 6월 5일 양자장론 문서 등재를 기점으로 총 등재 문서 수가 1만 개를 돌파하였고 이어 동해 11월에 제1회 정보트러스트 어워드 인터넷 문화 일반 분야에 선정되었다. 2007년 8월 9일에는 한겨레21에서 한국어 위키백과와 위키백과 오프라인 첫 모임을 취재한 기사를 표지 이야기로 다루었다.
2008년 광우병 촛불 시위 때 생긴 신조어인 명박산성이 한국어 위키백과에 등재되고 이 문서의 존치 여부를 두고 갑론을박의 과정이 화제가 되고 각종 매체에도 보도가 되었다. 시위대의 난입과 충돌을 방지하기 위해 거리에 설치되었던 컨테이너 박스를 이명박 정부의 불통으로 풍자를 하기 위해 사용된 이 신조어는 중립성을 지켰는지와 백과사전에 올라올 만한 문서인지가 쟁점이 되었는데 일시적으로 사용된 신조어일 뿐이라는 주장과 이미 여러 매체에서 사용되어 지속성이 보장되었다는 주장 등 논쟁이 벌어졌고 다음 아고라 등지에서 이 항목을 존치하는 방안을 지지하는 의견을 남기기 위해 여러 사람이 새로 가입하는 등 혼란이 빚어졌다. 11월 4일에는 다음커뮤니케이션에서 글로벌 세계 대백과사전을 기증받았으며, 2009년 3월에는 서울특별시로부터 콘텐츠를 기증받았다. 2009년 6월 4일에는 액세스권 등재를 기점으로 10만 개 문서 수를 돌파했다.
2011년 4월 16일에는 대한민국에서의 위키미디어 프로젝트를 지원하는 모임을 결성할 것을 추진하는 논의가 이뤄졌고 이후 창립준비위원회 결성을 거쳐 2014년 10월 19일 창립총회를 개최하였으며, 최종적으로 2015년 11월 4일 사단법인 한국 위키미디어 협회가 결성되어 활동 중에 있다. 2019년 미국 위키미디어재단으로부터 한국 지역 지부(챕터)로 승인을 받았다.
2012년 5월 19일에는 보비 탬블링 등재를 기점으로 총 20만 개 문서가 등재되었고 2015년 1월 5일, Rojo -Tierra- 문서 등재를 기점으로 총 30만 개 문서가 등재되었다. 2017년 10월 21일에는 충청남도 동물위생시험소 문서 등재로 40만 개의 문서까지 등재되었다."""

In [14]:
# wiki 내용 확인
with zipfile.ZipFile(os.path.join(kowiki_dir, 'kowiki.txt.zip')) as z:
    with z.open('kowiki.txt') as f:
        for i, line in enumerate(f):
            if i >= 100:
                break
            line = line.decode('utf-8').strip()
            print(line)

지미 카터
제임스 얼 "지미" 카터 주니어(, 1924년 10월 1일 ~ )는 민주당 출신 미국 39대 대통령 (1977년 ~ 1981년)이다.
지미 카터는 조지아주 섬터 카운티 플레인스 마을에서 태어났다. 조지아 공과대학교를 졸업하였다. 그 후 해군에 들어가 전함·원자력·잠수함의 승무원으로 일하였다. 1953년 미국 해군 대위로 예편하였고 이후 땅콩·면화 등을 가꿔 많은 돈을 벌었다. 그의 별명이 "땅콩 농부" (Peanut Farmer)로 알려졌다.
1962년 조지아 주 상원 의원 선거에서 낙선하나 그 선거가 부정선거 였음을 입증하게 되어 당선되고, 1966년 조지아 주 지사 선거에 낙선하지만 1970년 조지아 주 지사를 역임했다. 대통령이 되기 전 조지아주 상원의원을 두번 연임했으며, 1971년부터 1975년까지 조지아 지사로 근무했다. 조지아 주지사로 지내면서, 미국에 사는 흑인 등용법을 내세웠다.
1976년 미합중국 (미국) 제39대 대통령 선거에 민주당 후보로 출마하여 도덕주의 정책으로 내세워서, 많은 지지를 받고 제럴드 포드 (당시 미국 대통령) 를 누르고 당선되었다.
카터 대통령은 에너지 개발을 촉구했으나 공화당의 반대로 무산되었다.
카터는 이집트와 이스라엘을 조정하여, 캠프 데이비드에서 안와르 사다트 대통령과 메나헴 베긴 수상과 함께 중동 평화를 위한 캠프데이비드 협정을 체결했다.
그러나 이것은 공화당과 미국의 유대인 단체의 반발을 일으켰다. 1979년 백악관에서 양국 간의 평화조약으로 이끌어졌다. 또한 소련과 제2차 전략 무기 제한 협상에 조인했다.
카터는 1970년대 후반 당시 대한민국 등 인권 후진국의 국민들의 인권을 지키기 위해 노력했으며, 취임 이후 계속해서 도덕정치를 내세웠다.
그러나 주 이란 미국 대사관 인질 사건에서 인질 구출 실패를 이유로 1980년 대통령 선거에서 공화당의 로널드 레이건 후보에게 져 결국 재선에 실패했다. 또한 임기 말기에 터진 소련의 아프가니스탄 침공 사건으로 인해 1980년 하계 올림픽에 반공국가들의 보이콧

# Char Tokenizer

In [17]:
aa = collections.defaultdict(int)
aa['bb']
aa

defaultdict(int, {'bb': 0})

In [18]:
char_counter = collections.defaultdict(int)
# char 개수 확인
with zipfile.ZipFile(os.path.join(kowiki_dir, 'kowiki.txt.zip')) as z:
    with z.open('kowiki.txt') as f:
        for i, line in enumerate(f):
            if i >= 100000:
                break
            line = line.decode('utf-8').strip()
            for c in line:
                char_counter[c] += 1
len(char_counter)

6966

In [19]:
list(char_counter.items())[:100]

[('지', 123099),
 ('미', 25881),
 (' ', 3034409),
 ('카', 13719),
 ('터', 19622),
 ('제', 55447),
 ('임', 12950),
 ('스', 68934),
 ('얼', 1362),
 ('"', 21543),
 ('주', 62657),
 ('니', 19445),
 ('어', 73342),
 ('(', 69871),
 (',', 175903),
 ('1', 115707),
 ('9', 55493),
 ('2', 62377),
 ('4', 31403),
 ('년', 69536),
 ('0', 72055),
 ('월', 27540),
 ('일', 63346),
 ('~', 3517),
 (')', 69954),
 ('는', 220698),
 ('민', 27615),
 ('당', 27610),
 ('출', 11416),
 ('신', 32397),
 ('국', 68735),
 ('3', 36097),
 ('대', 100683),
 ('통', 24397),
 ('령', 9735),
 ('7', 28253),
 ('8', 33276),
 ('이', 312940),
 ('다', 279190),
 ('.', 227889),
 ('조', 35246),
 ('아', 72554),
 ('섬', 2519),
 ('운', 17477),
 ('티', 7305),
 ('플', 3806),
 ('레', 13160),
 ('인', 87587),
 ('마', 27668),
 ('을', 155879),
 ('에', 231250),
 ('서', 114998),
 ('태', 13110),
 ('났', 3043),
 ('공', 35992),
 ('과', 62245),
 ('학', 39986),
 ('교', 37519),
 ('를', 93642),
 ('졸', 861),
 ('업', 12978),
 ('하', 178051),
 ('였', 41292),
 ('그', 58693),
 ('후', 25082),
 ('해', 56091),
 ('군'

In [20]:
# 각 글자별 고유한 번호 부여
char_to_id = {'[PAD]': 0, '[UNK]': 1}
char_to_id

{'[PAD]': 0, '[UNK]': 1}

In [22]:
chr(120)

'x'

In [23]:
# Ascii 등록
index_s, index_e = 32, 126
for index in range(index_s, index_e + 1):
    char_to_id[chr(index)] = len(char_to_id)
len(char_to_id), list(char_to_id.items())[-20:]

(97,
 [('k', 97),
  ('l', 97),
  ('m', 97),
  ('n', 97),
  ('o', 97),
  ('p', 97),
  ('q', 97),
  ('r', 97),
  ('s', 97),
  ('t', 97),
  ('u', 97),
  ('v', 97),
  ('w', 97),
  ('x', 97),
  ('y', 97),
  ('z', 97),
  ('{', 97),
  ('|', 97),
  ('}', 97),
  ('~', 97)])

In [24]:
ord('가'), ord('힣'), chr(44032)

(44032, 55203, '가')

In [25]:
# 한글 등록
index_s, index_e = ord('가'), ord('힣')
for index in range(index_s, index_e + 1):
    char_to_id[chr(index)] = len(char_to_id)
len(char_to_id), list(char_to_id.items())[-20:]

(11269,
 [('힐', 11249),
  ('힑', 11250),
  ('힒', 11251),
  ('힓', 11252),
  ('힔', 11253),
  ('힕', 11254),
  ('힖', 11255),
  ('힗', 11256),
  ('힘', 11257),
  ('힙', 11258),
  ('힚', 11259),
  ('힛', 11260),
  ('힜', 11261),
  ('힝', 11262),
  ('힞', 11263),
  ('힟', 11264),
  ('힠', 11265),
  ('힡', 11266),
  ('힢', 11267),
  ('힣', 11268)])

In [26]:
# vocab 출력
print(f"전체: {len(char_to_id)}")
print(f"10개: {list(char_to_id.items())[:10]}")
print(f"alphabet: {list(char_to_id.items())[35:45]}")
print(f"한글처음: {list(char_to_id.items())[97:107]}")
print(f"한글마지막: {list(char_to_id.items())[-10:]}")

전체: 11269
10개: [('[PAD]', 0), ('[UNK]', 1), (' ', 97), ('!', 97), ('"', 97), ('#', 97), ('$', 97), ('%', 97), ('&', 97), ("'", 97)]
alphabet: [('A', 97), ('B', 97), ('C', 97), ('D', 97), ('E', 97), ('F', 97), ('G', 97), ('H', 97), ('I', 97), ('J', 97)]
한글처음: [('가', 97), ('각', 98), ('갂', 99), ('갃', 100), ('간', 101), ('갅', 102), ('갆', 103), ('갇', 104), ('갈', 105), ('갉', 106)]
한글마지막: [('힚', 11259), ('힛', 11260), ('힜', 11261), ('힝', 11262), ('힞', 11263), ('힟', 11264), ('힠', 11265), ('힡', 11266), ('힢', 11267), ('힣', 11268)]


In [27]:
# tokenize
char_tokens, char_ids = [], []
for c in text:
    if c == '\n':
        continue
    else:
        char_tokens.append(c)
        char_ids.append(char_to_id.get(c, 1))

# 결과 출력 (최초 64개만 출력)
print(char_tokens[:64])
print(char_ids[:64])

['위', '키', '백', '과', '의', ' ', '최', '상', '위', ' ', '도', '메', '인', '이', ' ', '.', 'c', 'o', 'm', '이', '던', ' ', '시', '절', ' ', 'k', 'o', '.', 'w', 'i', 'k', 'i', 'p', 'e', 'd', 'i', 'a', '.', 'c', 'o', 'm', '에', ' ', '구', '판', ' ', '미', '디', '어', '위', '키', '가', ' ', '깔', '렸', '으', '나', ' ', '한', '글', ' ', '처', '리', '에']
[7013, 9477, 4242, 349, 7097, 97, 8637, 5410, 7013, 97, 2085, 3765, 7129, 7125, 97, 97, 97, 97, 97, 7125, 1977, 97, 5949, 7273, 97, 97, 97, 97, 97, 97, 97, 97, 97, 97, 97, 97, 97, 97, 97, 97, 97, 6705, 97, 461, 10097, 97, 4185, 2421, 6677, 7013, 9477, 97, 97, 693, 3225, 7069, 1273, 97, 10685, 609, 97, 8441, 3597, 6705]


# Word Tokenizer

In [28]:
word_counter = collections.defaultdict(int)
# word 개수 확인
with zipfile.ZipFile(os.path.join(kowiki_dir, 'kowiki.txt.zip')) as z:
    with z.open('kowiki.txt') as f:
        for i, line in enumerate(f):
            if i >= 100000:
                break
            line = line.decode('utf-8').strip()
            for w in line.split():
                word_counter[w] += 1
len(word_counter)

687561

In [29]:
list(word_counter.items())[:100]

[('지미', 52),
 ('카터', 29),
 ('제임스', 253),
 ('얼', 4),
 ('"지미"', 1),
 ('주니어(,', 1),
 ('1924년', 124),
 ('10월', 1631),
 ('1일', 591),
 ('~', 1248),
 (')는', 623),
 ('민주당', 236),
 ('출신', 302),
 ('미국', 2285),
 ('39대', 1),
 ('대통령', 1189),
 ('(1977년', 1),
 ('1981년)이다.', 1),
 ('카터는', 15),
 ('조지아주', 12),
 ('섬터', 4),
 ('카운티', 20),
 ('플레인스', 1),
 ('마을에서', 35),
 ('태어났다.', 423),
 ('조지아', 34),
 ('공과대학교를', 2),
 ('졸업하였다.', 49),
 ('그', 11637),
 ('후', 2786),
 ('해군에', 15),
 ('들어가', 197),
 ('전함·원자력·잠수함의', 1),
 ('승무원으로', 1),
 ('일하였다.', 29),
 ('1953년', 145),
 ('해군', 156),
 ('대위로', 11),
 ('예편하였고', 1),
 ('이후', 5490),
 ('땅콩·면화', 1),
 ('등을', 2429),
 ('가꿔', 1),
 ('많은', 4731),
 ('돈을', 218),
 ('벌었다.', 11),
 ('그의', 5161),
 ('별명이', 30),
 ('"땅콩', 1),
 ('농부"', 1),
 ('(Peanut', 1),
 ('Farmer)로', 1),
 ('알려졌다.', 211),
 ('1962년', 196),
 ('주', 839),
 ('상원', 43),
 ('의원', 163),
 ('선거에서', 266),
 ('낙선하나', 1),
 ('선거가', 68),
 ('부정선거', 15),
 ('였음을', 1),
 ('입증하게', 2),
 ('되어', 1504),
 ('당선되고,', 3),
 ('1966년', 103),
 ('지사', 13),
 ('선거에'

In [30]:
# 각 단어별 고유한 번호 부여
word_to_id = {'[PAD]': 0, '[UNK]': 1}
word_to_id

{'[PAD]': 0, '[UNK]': 1}

In [31]:
# 단어 목록을 생성. set을 이용해 중복 제거
words = list(dict.fromkeys(text.split()))
print(f"words: {words}")

words: ['위키백과의', '최상위', '도메인이', '.com이던', '시절', 'ko.wikipedia.com에', '구판', '미디어위키가', '깔렸으나', '한글', '처리에', '문제가', '있어', '글을', '올릴', '수도', '없는', '이름뿐인', '곳이었다.', '2002년', '10월에', '새로운', '위키', '소프트웨어를', '쓰면서', '처리', '풀리기', '시작했지만,', '가장', '많은', '사람이', '쓰는', '인터넷', '익스플로러에서는', '인코딩', '여전했다.', '이런', '이유로', '초기에는', '도움말을', '옮기거나', '일에', '어려움을', '겪었다.', '어려움이', '있었는데도', '위키백과', '통계로는,', '10월에서', '2003년', '7월까지', '열', '달', '사이에', '글이', '13개에서', '159개로', '늘었고', '7월과', '8월', '사이에는', '한', '만에', '159개에서', '348개로', '늘어났다.', '9월부터는', '익스플로러의', '사라졌으며,', '대한민국', '언론에서도', '몇', '차례', '위키백과를', '소개하면서', '참여자가', '점증하리라고', '예측했다.', '참고로', '한국어', '최초', '문서는', '10월', '12일에', '등재된', '지미', '카터', '문서이다.', '2005년', '6월', '5일', '양자장론', '문서', '등재를', '기점으로', '총', '등재', '수가', '1만', '개를', '돌파하였고', '이어', '동해', '11월에', '제1회', '정보트러스트', '어워드', '문화', '일반', '분야에', '선정되었다.', '2007년', '9일에는', '한겨레21에서', '위키백과와', '오프라인', '첫', '모임을', '취재한', '기사를', '표지', '이야기로', '다루었다.', '2008년', '광우병', '촛불', '시위', '때', '생긴', '신조어인', '명박산성이', 

In [32]:
# 단어를 vocab에 등록
for word in words:
    word_to_id[word] = len(word_to_id)

In [33]:
# vocab 개수 출력
print(f"전체: {len(word_to_id)}")
print(f"처음 10개: {list(word_to_id.items())[:10]}")
print(f"마지막 10개: {list(word_to_id.items())[-10:]}")

전체: 275
처음 10개: [('[PAD]', 0), ('[UNK]', 1), ('위키백과의', 2), ('최상위', 3), ('도메인이', 4), ('.com이던', 5), ('시절', 6), ('ko.wikipedia.com에', 7), ('구판', 8), ('미디어위키가', 9)]
마지막 10개: [('30만', 265), ('등재되었다.', 266), ('2017년', 267), ('21일에는', 268), ('충청남도', 269), ('동물위생시험소', 270), ('등재로', 271), ('40만', 272), ('개의', 273), ('문서까지', 274)]


In [34]:
# tokenize
word_tokens, word_ids = [], []
for word in text.split():
    word_tokens.append(word)
    word_ids.append(word_to_id.get(word, 1))

# 결과 출력 (최초 64개만 출력)
print(word_tokens[:64])
print(word_ids[:64])

['위키백과의', '최상위', '도메인이', '.com이던', '시절', 'ko.wikipedia.com에', '구판', '미디어위키가', '깔렸으나', '한글', '처리에', '문제가', '있어', '글을', '올릴', '수도', '없는', '이름뿐인', '곳이었다.', '2002년', '10월에', '새로운', '위키', '소프트웨어를', '쓰면서', '한글', '처리', '문제가', '풀리기', '시작했지만,', '가장', '많은', '사람이', '쓰는', '인터넷', '익스플로러에서는', '인코딩', '문제가', '여전했다.', '이런', '이유로', '초기에는', '도움말을', '옮기거나', '쓰는', '일에', '어려움을', '겪었다.', '이런', '어려움이', '있었는데도', '위키백과', '통계로는,', '2002년', '10월에서', '2003년', '7월까지', '열', '달', '사이에', '글이', '13개에서', '159개로', '늘었고']
[2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 11, 27, 13, 28, 29, 30, 31, 32, 33, 34, 35, 36, 13, 37, 38, 39, 40, 41, 42, 33, 43, 44, 45, 38, 46, 47, 48, 49, 21, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59]


# BPE (Byte Pair Encoding)

In [35]:
# 최초 말뭉치 빈도수
bpe_counter = {'_ l o w': 5,
         '_ l o w e r': 2,
         '_ n e w e s t': 6,
         '_ w i d e s t': 3
         }

In [36]:
# 각 subword에 고유한 번호 부여
bpe_to_id = {'[PAD]': 0, '[UNK]': 1}
bpe_to_id

{'[PAD]': 0, '[UNK]': 1}

In [37]:
def get_vocab(counter, vocab):
    for word, freq in counter.items():
        tokens = word.split()
        for token in tokens:
            if token not in vocab:
                vocab[token] = len(bpe_to_id)
    return vocab

In [38]:
bpe_to_id = get_vocab(bpe_counter, bpe_to_id)
len(bpe_to_id), bpe_to_id

(13,
 {'[PAD]': 0,
  '[UNK]': 1,
  '_': 2,
  'd': 12,
  'e': 6,
  'i': 11,
  'l': 3,
  'n': 8,
  'o': 4,
  'r': 7,
  's': 9,
  't': 10,
  'w': 5})

In [39]:
def get_bi_gram(counter):
    """
    bi-gram 횟수를 구하는 함수
    :param counter: bpe counter
    :return: bi-gram 빈도수 dictionary
    """
    pairs = collections.defaultdict(int)  # 새로운 단어는 기본 값 0
    for word, freq in counter.items():
        tokens = word.split() # 값은 띄어쓰기로 구분 되어 있음 'l o v e'
        for i in range(len(tokens) - 1):
            pairs[(tokens[i], tokens[i + 1])] += freq # 이전 단어와 다음 단어의 빈도수
    return pairs

In [40]:
pairs = get_bi_gram(bpe_counter)
pairs

defaultdict(int,
            {('_', 'l'): 7,
             ('_', 'n'): 6,
             ('_', 'w'): 3,
             ('d', 'e'): 3,
             ('e', 'r'): 2,
             ('e', 's'): 9,
             ('e', 'w'): 6,
             ('i', 'd'): 3,
             ('l', 'o'): 7,
             ('n', 'e'): 6,
             ('o', 'w'): 7,
             ('s', 't'): 9,
             ('w', 'e'): 8,
             ('w', 'i'): 3})

In [41]:
best = max(pairs, key=pairs.get)  # value 이 가장 큰 pair 조회
best

('e', 's')

In [48]:
re.escape(' '.join(best))

'e\\ s'

In [42]:
def merge_counter(pair, counter_in):
    """
    bi-gram을 합치는 함수
    :param pair: bi-gram pair
    :param counter_in: 현재 bpe counter
    :return: bi-gram이 합쳐진 새로운 counter
    """
    counter_out = {}
    # 두 단어를 의미하는 regex 생성
    bigram = re.escape(' '.join(pair))
    # not a whitespace character: \S
    # negative lookbehind assertion: (?<!...)
    # negative lookahead assertion: (?!...)
    p = re.compile(r'(?<!\S)' + bigram + r'(?!\S)')
    unigram = ''.join(pair)
    print(f'bigram: {bigram} -> unigram: {unigram}')
    for word in counter_in:
        w_out = p.sub(unigram, word)  # bigram을 unigram으로 변경
        counter_out[w_out] = counter_in[word]
    return counter_out

In [50]:
bpe_counter, best

({'_ l o w': 5, '_ l o w e r': 2, '_ n e w es t': 6, '_ w i d es t': 3},
 ('e', 's'))

In [51]:
merge_counter(best, bpe_counter)

bigram: e\ s -> unigram: es


{'_ l o w': 5, '_ l o w e r': 2, '_ n e w es t': 6, '_ w i d es t': 3}

In [52]:
for i in range(1):
    pairs = get_bi_gram(bpe_counter)
    print("pairs:", pairs)
    best = max(pairs, key=pairs.get)  # value 이 가장 큰 pair 조회
    print("best:", best)
    bpe_counter = merge_counter(best, bpe_counter)
    print("counter:", bpe_counter)
    bpe_to_id = get_vocab(bpe_counter, bpe_to_id)
    print("vocab:", bpe_to_id)

pairs: defaultdict(<class 'int'>, {('_', 'l'): 7, ('l', 'o'): 7, ('o', 'w'): 7, ('w', 'e'): 2, ('e', 'r'): 2, ('_', 'n'): 6, ('n', 'e'): 6, ('e', 'w'): 6, ('w', 'es'): 6, ('es', 't'): 9, ('_', 'w'): 3, ('w', 'i'): 3, ('i', 'd'): 3, ('d', 'es'): 3})
best: ('es', 't')
bigram: es\ t -> unigram: est
counter: {'_ l o w': 5, '_ l o w e r': 2, '_ n e w est': 6, '_ w i d est': 3}
vocab: {'[PAD]': 0, '[UNK]': 1, '_': 2, 'l': 3, 'o': 4, 'w': 5, 'e': 6, 'r': 7, 'n': 8, 's': 9, 't': 10, 'i': 11, 'd': 12, 'es': 13, 'est': 14}


# Google sentencepiece를 이용해 vocab 생성
- https://github.com/google/sentencepiece

## sentencepe 학습

In [53]:
os.listdir(kowiki_dir)

['kowiki.txt.zip']

In [54]:
shutil.copy(os.path.join(kowiki_dir, 'kowiki.txt.zip'), './')
os.listdir('./')

['.config', 'drive', 'kowiki.txt.zip', 'sample_data']

In [68]:
!unzip kowiki.txt.zip
os.listdir('./')

Archive:  kowiki.txt.zip
  inflating: kowiki.txt              

['.config', 'drive', 'kowiki.txt.zip', 'kowiki.txt', 'sample_data']

In [69]:
def train_sentencepiece(corpus, prefix, vocab_size=32000):
    """
    sentencepiece를 이용해 vocab 학습
    :param corpus: 학습할 말뭉치
    :param prefix: 저장할 vocab 이름
    :param vocab_size: vocab 개수
    """
    spm.SentencePieceTrainer.train(
        f"--input={corpus} --model_prefix={prefix} --vocab_size={vocab_size + 7}" +  # 7은 특수문자 개수
        " --model_type=unigram" +
        " --max_sentence_length=999999" +  # 문장 최대 길이
        " --pad_id=0 --pad_piece=[PAD]" +  # pad token 및 id 지정
        " --unk_id=1 --unk_piece=[UNK]" +  # unknown token 및 id 지정
        " --bos_id=2 --bos_piece=[BOS]" +  # begin of sequence token 및 id 지정
        " --eos_id=3 --eos_piece=[EOS]" +  # end of sequence token 및 id 지정
        " --user_defined_symbols=[SEP],[CLS],[MASK]")  # 기타 추가 토큰 SEP: 4, CLS: 5, MASK: 6

In [70]:
# vocab 생성
train_sentencepiece(f"kowiki.txt", f"ko_32000", vocab_size=32000)

In [73]:
os.listdir(".")

['.config',
 'drive',
 'kowiki.txt.zip',
 'kowiki.txt',
 'ko_32000.model',
 'ko_32000.vocab',
 'sample_data']

In [74]:
# 생성된 vocab 복사
shutil.copy("ko_32000.model", f"{data_dir}/ko_32000.model")
shutil.copy("ko_32000.vocab", f"{data_dir}/ko_32000.vocab")
os.listdir(f"{data_dir}")

['Pandas 튜토리얼.ipynb',
 'Matplotlib 튜토리얼.ipynb의 사본',
 'Day1_EVN.ipynb의 사본',
 'Day1_Assignment.ipynb',
 'ko_32000.model',
 'ko_32000.vocab',
 'Day2_01_Encoding',
 'Day2_02_Tokenizer',
 'kowiki',
 'kowiki.txt',
 'Untitled Folder',
 '.ipynb_checkpoints']

## sentencepe 확인

In [55]:
# load vocab
spm_vocab = spm.SentencePieceProcessor()
spm_vocab.load(f"{data_dir}/ko_32000.model")

True

In [56]:
# vocab 출력
print(f"len: {len(spm_vocab)}")
for id in range(16):
    print(f"{id:2d}: {spm_vocab.id_to_piece(id)}")

len: 32007
 0: [PAD]
 1: [UNK]
 2: [BOS]
 3: [EOS]
 4: [SEP]
 5: [CLS]
 6: [MASK]
 7: .
 8: ,
 9: 의
10: ▁
11: 는
12: )
13: 에
14: (
15: 년


In [57]:
# text를 tokenize 함
# sentence to pieces
###############################
pieces = spm_vocab.encode_as_pieces(text)
print(pieces)
###############################

['▁위키백과', '의', '▁최상위', '▁도메인', '이', '▁', '.', 'com', '이던', '▁시절', '▁', 'ko', '.', 'w', 'iki', 'pe', 'dia', '.', 'com', '에', '▁구', '판', '▁미디어', '위', '키', '가', '▁깔', '렸으나', '▁한글', '▁처리', '에', '▁문제가', '▁있어', '▁글을', '▁올릴', '▁수도', '▁없는', '▁이름', '뿐', '인', '▁곳이었다', '.', '▁2002', '년', '▁10', '월에', '▁새로운', '▁위키', '▁소프트웨어를', '▁쓰', '면서', '▁한글', '▁처리', '▁문제가', '▁풀리', '기', '▁시작', '했지만', ',', '▁가장', '▁많은', '▁사람이', '▁쓰는', '▁인터넷', '▁익스플로러', '에서는', '▁인코딩', '▁문제가', '▁여', '전', '했다', '.', '▁이런', '▁이유로', '▁초기에는', '▁도움', '말', '을', '▁옮기', '거나', '▁쓰는', '▁', '일에', '▁어려움', '을', '▁겪었다', '.', '▁이런', '▁어려움이', '▁있었는데', '도', '▁위키백과', '▁통계', '로', '는', ',', '▁2002', '년', '▁10', '월', '에서', '▁2003', '년', '▁7', '월까지', '▁열', '▁달', '▁사이에', '▁글', '이', '▁13', '개', '에서', '▁1', '59', '개로', '▁늘', '었고', '▁2003', '년', '▁7', '월', '과', '▁8', '월', '▁사이에는', '▁한', '▁달', '▁만에', '▁1', '59', '개', '에서', '▁3', '48', '개로', '▁늘어났다', '.', '▁2003', '년', '▁9', '월부터', '는', '▁인터넷', '▁익스플로러', '의', '▁인코딩', '▁문제가', '▁사라', '졌으며', ',', '▁대한민국', '▁언론',

In [58]:
###############################
spm_vocab.encode_as_pieces("나는 오늘 수원에 놀러 갔다.")
###############################

['▁나는', '▁오늘', '▁수원', '에', '▁놀', '러', '▁갔다', '.']

In [59]:
###############################
spm_vocab.encode_as_pieces("자연어처리어렵지만재미있다.")
###############################

['▁자연', '어', '처리', '어', '렵', '지만', '재', '미', '있다', '.']

In [60]:
# tokenize된 값을 string 으로 복원
# pieces to sentence
###############################
spm_vocab.decode_pieces(pieces)
###############################

'위키백과의 최상위 도메인이 .com이던 시절 ko.wikipedia.com에 구판 미디어위키가 깔렸으나 한글 처리에 문제가 있어 글을 올릴 수도 없는 이름뿐인 곳이었다. 2002년 10월에 새로운 위키 소프트웨어를 쓰면서 한글 처리 문제가 풀리기 시작했지만, 가장 많은 사람이 쓰는 인터넷 익스플로러에서는 인코딩 문제가 여전했다. 이런 이유로 초기에는 도움말을 옮기거나 쓰는 일에 어려움을 겪었다. 이런 어려움이 있었는데도 위키백과 통계로는, 2002년 10월에서 2003년 7월까지 열 달 사이에 글이 13개에서 159개로 늘었고 2003년 7월과 8월 사이에는 한 달 만에 159개에서 348개로 늘어났다. 2003년 9월부터는 인터넷 익스플로러의 인코딩 문제가 사라졌으며, 대한민국 언론에서도 몇 차례 위키백과를 소개하면서 참여자가 점증하리라고 예측했다. 참고로 한국어 위키백과의 최초 문서는 2002년 10월 12일에 등재된 지미 카터 문서이다. 2005년 6월 5일 양자장론 문서 등재를 기점으로 총 등재 문서 수가 1만 개를 돌파하였고 이어 동해 11월에 제1회 정보트러스트 어워드 인터넷 문화 일반 분야에 선정되었다. 2007년 8월 9일에는 한겨레21에서 한국어 위키백과와 위키백과 오프라인 첫 모임을 취재한 기사를 표지 이야기로 다루었다. 2008년 광우병 촛불 시위 때 생긴 신조어인 명박산성이 한국어 위키백과에 등재되고 이 문서의 존치 여부를 두고 갑론을박의 과정이 화제가 되고 각종 매체에도 보도가 되었다. 시위대의 난입과 충돌을 방지하기 위해 거리에 설치되었던 컨테이너 박스를 이명박 정부의 불통으로 풍자를 하기 위해 사용된 이 신조어는 중립성을 지켰는지와 백과사전에 올라올 만한 문서인지가 쟁점이 되었는데 일시적으로 사용된 신조어일 뿐이라는 주장과 이미 여러 매체에서 사용되어 지속성이 보장되었다는 주장 등 논쟁이 벌어졌고 다음 아고라 등지에서 이 항목을 존치하는 방안을 지지하는 의견을 남기기 위해 여러 사람이 새로 가입하는 등 혼란이 빚어졌다. 11월 4일

In [64]:
# tokenize된 값을 id로 변경
# piece to id
piece_ids = []
for piece in pieces:
    ########################
    _id = spm_vocab.piece_to_id(piece)
    piece_ids.append(_id)
    ########################
print(piece_ids[0:100])

[10603, 9, 10040, 7746, 17, 10, 7, 4346, 3228, 1239, 10, 9746, 7, 1813, 26734, 4986, 15537, 7, 4346, 13, 206, 568, 2794, 209, 323, 19, 13305, 13104, 4626, 1581, 13, 1910, 499, 3772, 22984, 588, 500, 1587, 4285, 31, 21149, 7, 780, 15, 67, 436, 418, 13323, 16652, 1711, 456, 4626, 1581, 1910, 22657, 48, 949, 703, 8, 139, 192, 1183, 3460, 1412, 20619, 118, 21038, 1910, 604, 145, 53, 7, 1001, 1143, 5997, 14360, 827, 16, 7977, 847, 3460, 10, 193, 6032, 16, 10408, 7, 1001, 22453, 2781, 32, 10603, 5696, 21, 11, 8, 780, 15, 67, 23]


In [65]:
# text를 id로 tokenize 함
# sentence to ids
###############################
ids = spm_vocab.encode_as_ids(text)
print(ids)
###############################

[10603, 9, 10040, 7746, 17, 10, 7, 4346, 3228, 1239, 10, 9746, 7, 1813, 26734, 4986, 15537, 7, 4346, 13, 206, 568, 2794, 209, 323, 19, 13305, 13104, 4626, 1581, 13, 1910, 499, 3772, 22984, 588, 500, 1587, 4285, 31, 21149, 7, 780, 15, 67, 436, 418, 13323, 16652, 1711, 456, 4626, 1581, 1910, 22657, 48, 949, 703, 8, 139, 192, 1183, 3460, 1412, 20619, 118, 21038, 1910, 604, 145, 53, 7, 1001, 1143, 5997, 14360, 827, 16, 7977, 847, 3460, 10, 193, 6032, 16, 10408, 7, 1001, 22453, 2781, 32, 10603, 5696, 21, 11, 8, 780, 15, 67, 23, 22, 800, 15, 74, 2026, 713, 783, 749, 2266, 17, 286, 119, 22, 35, 3428, 6751, 3151, 1623, 800, 15, 74, 23, 24, 73, 23, 9307, 59, 783, 1674, 35, 3428, 119, 22, 38, 2074, 6751, 14982, 7, 800, 15, 83, 1327, 11, 1412, 20619, 9, 21038, 1910, 4947, 6217, 8, 255, 2060, 664, 740, 2653, 10603, 20, 2239, 248, 1758, 962, 1026, 756, 28793, 37, 3517, 53, 7, 10100, 3977, 10603, 9, 3384, 10991, 780, 15, 67, 23, 89, 193, 18285, 85, 14473, 14639, 2061, 30, 7, 629, 15, 64, 23, 51, 26,

In [66]:
# tokenize된 id 값을 string 으로 복원
# id to sentence
###############################
spm_vocab.decode_ids(ids)
###############################

'위키백과의 최상위 도메인이 .com이던 시절 ko.wikipedia.com에 구판 미디어위키가 깔렸으나 한글 처리에 문제가 있어 글을 올릴 수도 없는 이름뿐인 곳이었다. 2002년 10월에 새로운 위키 소프트웨어를 쓰면서 한글 처리 문제가 풀리기 시작했지만, 가장 많은 사람이 쓰는 인터넷 익스플로러에서는 인코딩 문제가 여전했다. 이런 이유로 초기에는 도움말을 옮기거나 쓰는 일에 어려움을 겪었다. 이런 어려움이 있었는데도 위키백과 통계로는, 2002년 10월에서 2003년 7월까지 열 달 사이에 글이 13개에서 159개로 늘었고 2003년 7월과 8월 사이에는 한 달 만에 159개에서 348개로 늘어났다. 2003년 9월부터는 인터넷 익스플로러의 인코딩 문제가 사라졌으며, 대한민국 언론에서도 몇 차례 위키백과를 소개하면서 참여자가 점증하리라고 예측했다. 참고로 한국어 위키백과의 최초 문서는 2002년 10월 12일에 등재된 지미 카터 문서이다. 2005년 6월 5일 양자장론 문서 등재를 기점으로 총 등재 문서 수가 1만 개를 돌파하였고 이어 동해 11월에 제1회 정보트러스트 어워드 인터넷 문화 일반 분야에 선정되었다. 2007년 8월 9일에는 한겨레21에서 한국어 위키백과와 위키백과 오프라인 첫 모임을 취재한 기사를 표지 이야기로 다루었다. 2008년 광우병 촛불 시위 때 생긴 신조어인 명박산성이 한국어 위키백과에 등재되고 이 문서의 존치 여부를 두고 갑론을박의 과정이 화제가 되고 각종 매체에도 보도가 되었다. 시위대의 난입과 충돌을 방지하기 위해 거리에 설치되었던 컨테이너 박스를 이명박 정부의 불통으로 풍자를 하기 위해 사용된 이 신조어는 중립성을 지켰는지와 백과사전에 올라올 만한 문서인지가 쟁점이 되었는데 일시적으로 사용된 신조어일 뿐이라는 주장과 이미 여러 매체에서 사용되어 지속성이 보장되었다는 주장 등 논쟁이 벌어졌고 다음 아고라 등지에서 이 항목을 존치하는 방안을 지지하는 의견을 남기기 위해 여러 사람이 새로 가입하는 등 혼란이 빚어졌다. 11월 4일

In [67]:
# id 값을 token으로 변경
# id to piece
id_pieces = []
for id in ids:
    ########################
    piece = spm_vocab.id_to_piece(id)
    id_pieces.append(pieces)
    ########################
print(id_pieces[0:100])

[['▁위키백과', '의', '▁최상위', '▁도메인', '이', '▁', '.', 'com', '이던', '▁시절', '▁', 'ko', '.', 'w', 'iki', 'pe', 'dia', '.', 'com', '에', '▁구', '판', '▁미디어', '위', '키', '가', '▁깔', '렸으나', '▁한글', '▁처리', '에', '▁문제가', '▁있어', '▁글을', '▁올릴', '▁수도', '▁없는', '▁이름', '뿐', '인', '▁곳이었다', '.', '▁2002', '년', '▁10', '월에', '▁새로운', '▁위키', '▁소프트웨어를', '▁쓰', '면서', '▁한글', '▁처리', '▁문제가', '▁풀리', '기', '▁시작', '했지만', ',', '▁가장', '▁많은', '▁사람이', '▁쓰는', '▁인터넷', '▁익스플로러', '에서는', '▁인코딩', '▁문제가', '▁여', '전', '했다', '.', '▁이런', '▁이유로', '▁초기에는', '▁도움', '말', '을', '▁옮기', '거나', '▁쓰는', '▁', '일에', '▁어려움', '을', '▁겪었다', '.', '▁이런', '▁어려움이', '▁있었는데', '도', '▁위키백과', '▁통계', '로', '는', ',', '▁2002', '년', '▁10', '월', '에서', '▁2003', '년', '▁7', '월까지', '▁열', '▁달', '▁사이에', '▁글', '이', '▁13', '개', '에서', '▁1', '59', '개로', '▁늘', '었고', '▁2003', '년', '▁7', '월', '과', '▁8', '월', '▁사이에는', '▁한', '▁달', '▁만에', '▁1', '59', '개', '에서', '▁3', '48', '개로', '▁늘어났다', '.', '▁2003', '년', '▁9', '월부터', '는', '▁인터넷', '▁익스플로러', '의', '▁인코딩', '▁문제가', '▁사라', '졌으며', ',', '▁대한민국', '▁언론'

# 형태소 분석기

In [91]:
# 형태소분석기 설치
!set -x \
&& pip install konlpy \
&& curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh | bash -x

+ pip install konlpy
+ bash -x
+ curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh
+ mecab_dicdir=/usr/local/lib/mecab/dic/mecab-ko-dic
+ set -e
++ uname
+ os=Linux
+ [[ ! Linux == \L\i\n\u\x ]]
+ hash sudo
+ sudo=sudo
+ python=python3
+ hash pyenv
+ at_user_site=
++ check_python_site_location_is_writable
++ python3 -
+ [[ 1 == \0 ]]
+ hash automake
+ echo 'Installing automake (A dependency for mecab-ko)'
Installing automake (A dependency for mecab-ko)
+ install_automake
+ '[' Linux == Linux ']'
++ grep -Ei 'debian|buntu|mint' /etc/lsb-release /etc/os-release
+ '[' '/etc/lsb-release:DISTRIB_ID=Ubuntu
/etc/lsb-release:DISTRIB_DESCRIPTION="Ubuntu 18.04.5 LTS"
/etc/os-release:NAME="Ubuntu"
/etc/os-release:ID=ubuntu
/etc/os-release:ID_LIKE=debian
/etc/os-release:PRETTY_NAME="Ubuntu 18.04.5 LTS"
/etc/os-release:HOME_URL="https://www.ubuntu.com/"
/etc/os-release:SUPPORT_URL="https://help.ubuntu.com/"
/etc/os-release:BUG_REPORT_URL="https://bugs.launchpad.net/ubu

In [94]:
import konlpy

In [109]:
okt = konlpy.tag.Okt()
okt.morphs(text)

['위키',
 '백',
 '과의',
 '최',
 '상위',
 '도메인',
 '이',
 '.',
 'com',
 '이던',
 '시절',
 'ko.wikipedia.com',
 '에',
 '구판',
 '미디어위키',
 '가',
 '깔렸으나',
 '한글',
 '처리',
 '에',
 '문제',
 '가',
 '있어',
 '글',
 '을',
 '올릴',
 '수도',
 '없는',
 '이름',
 '뿐',
 '인',
 '곳',
 '이었다',
 '.',
 '2002년',
 '10월',
 '에',
 '새로운',
 '위키',
 '소프트웨어',
 '를',
 '쓰면서',
 '한글',
 '처리',
 '문제',
 '가',
 '풀리기',
 '시작',
 '했지만',
 ',',
 '가장',
 '많은',
 '사람',
 '이',
 '쓰는',
 '인터넷',
 '익스플로러',
 '에서는',
 '인코딩',
 '문제',
 '가',
 '여전했다',
 '.',
 '이런',
 '이유',
 '로',
 '초기',
 '에는',
 '도움말',
 '을',
 '옮기거나',
 '쓰는',
 '일',
 '에',
 '어려움',
 '을',
 '겪었다',
 '.',
 '이런',
 '어려움',
 '이',
 '있었는데도',
 '위키',
 '백',
 '과',
 '통계',
 '로는',
 ',',
 '2002년',
 '10월',
 '에서',
 '2003년',
 '7월',
 '까지',
 '열',
 '달',
 '사이',
 '에',
 '글',
 '이',
 '13',
 '개',
 '에서',
 '159',
 '개',
 '로',
 '늘었고',
 '2003년',
 '7월',
 '과',
 '8월',
 '사이',
 '에는',
 '한',
 '달',
 '만에',
 '159',
 '개',
 '에서',
 '348',
 '개',
 '로',
 '늘어났다',
 '.',
 '2003년',
 '9월',
 '부터는',
 '인터넷',
 '익스플로러',
 '의',
 '인코딩',
 '문제',
 '가',
 '사라졌으며',
 ',',
 '대한민국',
 '언론',
 '에서도',
 '

In [110]:
import MeCab
mecab = MeCab.Tagger()
print(mecab.parse(text))

위키백과	NNP,*,F,위키백과,Compound,*,*,위키/NNP/지명+백과/NNG/*
의	JKG,*,F,의,*,*,*,*
최상위	NNG,*,F,최상위,Compound,*,*,최/NNG/*+상위/NNG/정적사태
도메인	NNG,*,T,도메인,*,*,*,*
이	JKS,*,F,이,*,*,*,*
.	SY,*,*,*,*,*,*,*
com	SL,*,*,*,*,*,*,*
이	VCP,*,F,이,*,*,*,*
던	ETM,*,T,던,*,*,*,*
시절	NNG,*,T,시절,*,*,*,*
ko	SL,*,*,*,*,*,*,*
.	SY,*,*,*,*,*,*,*
wikipedia	SL,*,*,*,*,*,*,*
.	SY,*,*,*,*,*,*,*
com	SL,*,*,*,*,*,*,*
에	JKB,*,F,에,*,*,*,*
구판	NNG,*,T,구판,*,*,*,*
미디어위키	NNP,*,F,미디어위키,*,*,*,*
가	JKS,*,F,가,*,*,*,*
깔렸	VV+EP,*,T,깔렸,Inflect,VV,EP,깔리/VV/*+었/EP/*
으나	EC,*,F,으나,*,*,*,*
한글	NNG,*,T,한글,*,*,*,*
처리	NNG,행위,F,처리,*,*,*,*
에	JKB,*,F,에,*,*,*,*
문제	NNG,*,F,문제,*,*,*,*
가	JKS,*,F,가,*,*,*,*
있	VA,*,T,있,*,*,*,*
어	EC,*,F,어,*,*,*,*
글	NNG,*,T,글,*,*,*,*
을	JKO,*,T,을,*,*,*,*
올릴	VV+ETM,*,T,올릴,Inflect,VV,ETM,올리/VV/*+ᆯ/ETM/*
수	NNB,*,F,수,*,*,*,*
도	JX,*,F,도,*,*,*,*
없	VA,*,T,없,*,*,*,*
는	ETM,*,T,는,*,*,*,*
이름	NNG,*,T,이름,*,*,*,*
뿐	JX,*,T,뿐,*,*,*,*
인	VCP+ETM,*,T,인,Inflect,VCP,ETM,이/VCP/*+ᆫ/ETM/*
곳	NNG,*,T,곳,*,*,*,*
이	VCP,*,F,이,*,*,*,*
었	EP,*,T,었,*,*,*,*
다	EF,*,F,다,*,*

In [111]:
hannanum = konlpy.tag.Hannanum()
hannanum.morphs(text)

['위키백과',
 '의',
 '최상위',
 '도메인',
 '이',
 '.',
 'com',
 '이',
 '던',
 '시절',
 'ko',
 '.',
 'wikipedia',
 '.',
 'com',
 '에',
 '구판',
 '미디어위키',
 '가',
 '깔리',
 '었으나',
 '한글',
 '처리',
 '에',
 '문제',
 '가',
 '있',
 '어',
 '글',
 '을',
 '올리',
 'ㄹ',
 '수',
 '도',
 '없',
 '는',
 '이름',
 '뿐',
 '이',
 'ㄴ',
 '곳',
 '이',
 '었다',
 '.',
 '2002년',
 '10월',
 '에',
 '새롭',
 '은',
 '위키',
 '소프트웨어',
 '를',
 '쓰',
 '면서',
 '한글',
 '처리',
 '문제',
 '가',
 '풀리',
 '기',
 '시작',
 '하',
 '었지만',
 ',',
 '가장',
 '많',
 '은',
 '사람',
 '이',
 '쓰',
 '는',
 '인터넷',
 '익스플로러',
 '에서는',
 '인코딩',
 '문제',
 '가',
 '여전',
 '하',
 '었다',
 '.',
 '이런',
 '이유',
 '로',
 '초',
 '이',
 '기',
 '에는',
 '도움말',
 '을',
 '옮기',
 '거나',
 '쓰',
 '는',
 '일',
 '에',
 '어려움',
 '을',
 '겪',
 '었다',
 '.',
 '이런',
 '어려움',
 '이',
 '있',
 '었는데',
 '도',
 '위키백',
 '과',
 '통계',
 '로는',
 ',',
 '2002년',
 '10월',
 '에서',
 '2003년',
 '7월',
 '까지',
 '열',
 '달',
 '사이',
 '에',
 '글',
 '이',
 '13개',
 '에서',
 '159개',
 '로',
 '늘',
 '었고',
 '2003년',
 '7월',
 '과',
 '8월',
 '사이',
 '에는',
 '하',
 'ㄴ',
 '달',
 '만',
 '에',
 '159개',
 '에서',
 '348개',
 '로',
 '늘',

In [112]:
komoran = konlpy.tag.Komoran()
komoran.morphs(text)

['위키백과',
 '의',
 '최상위 도메인',
 '이',
 '.com',
 '이',
 '던',
 '시절',
 'ko',
 '.',
 'wikipedia',
 '.com',
 '에',
 '구판',
 '미디어위키',
 '가',
 '깔리',
 '었',
 '으나',
 '한글',
 '처리',
 '에',
 '문제',
 '가',
 '있',
 '어',
 '글',
 '을',
 '올리',
 'ㄹ',
 '수',
 '도',
 '없',
 '는',
 '이름',
 '뿐',
 '이',
 'ㄴ',
 '곳',
 '이',
 '었',
 '다',
 '.',
 '2002년 10월',
 '에',
 '새롭',
 'ㄴ',
 '위키 소프트웨어',
 '를',
 '쓰',
 '면서',
 '한글',
 '처리',
 '문제',
 '가',
 '풀리',
 '기',
 '시작',
 '하',
 '았',
 '지만',
 ',',
 '가장',
 '많',
 '은',
 '사람',
 '이',
 '쓰',
 '는',
 '인터넷 익스플로러',
 '에서',
 '는',
 '인코딩',
 '문제',
 '가',
 '여전',
 '하',
 '었',
 '다',
 '.',
 '이런',
 '이유',
 '로',
 '초기',
 '에',
 '는',
 '도움말',
 '을',
 '옮기',
 '거나',
 '쓰',
 '는',
 '일',
 '에',
 '어려움',
 '을',
 '겪',
 '었',
 '다',
 '.',
 '이런',
 '어려움',
 '이',
 '있',
 '었',
 '는데',
 '도',
 '위키백과',
 '통계',
 '로',
 '는',
 ',',
 '2002년 10월',
 '에서',
 '2003년 7월',
 '까지',
 '열',
 '달',
 '사이',
 '에',
 '글',
 '이',
 '13',
 '개',
 '에서',
 '159',
 '개',
 '로',
 '늘',
 '었',
 '고',
 '2003년 7월',
 '과',
 '8월',
 '사이',
 '에',
 '는',
 '한',
 '달',
 '만',
 '에',
 '159',
 '개',
 '에서',
 '348',
 '

In [113]:
kkma = konlpy.tag.Kkma()
kkma.morphs(text)

['위',
 '키',
 '백과',
 '의',
 '최상',
 '위',
 '도메인',
 '이',
 '.',
 'com',
 '이',
 '덜',
 'ㄴ',
 '시절',
 'ko',
 '.',
 'wikipedia',
 '.',
 'com',
 '에',
 '구판',
 '미디어',
 '위',
 '키',
 '가',
 '깔리',
 '었',
 '으나',
 '한글',
 '처리',
 '에',
 '문제',
 '가',
 '있',
 '어',
 '글',
 '을',
 '올리',
 'ㄹ',
 '수',
 '도',
 '없',
 '는',
 '이름',
 '뿐',
 '이',
 'ㄴ',
 '곳',
 '이',
 '었',
 '다',
 '.',
 '2002',
 '년',
 '10',
 '월',
 '에',
 '새로',
 '운',
 '위',
 '키',
 '소프트웨어',
 '를',
 '쓰',
 '면서',
 '한글',
 '처리',
 '문제',
 '가',
 '풀리',
 '기',
 '시작',
 '하',
 '었',
 '지만',
 ',',
 '가장',
 '많',
 '은',
 '사람',
 '이',
 '쓰',
 '는',
 '인터넷',
 '익스플로러',
 '에서',
 '는',
 '인',
 '코딩',
 '문제',
 '가',
 '여전',
 '하',
 '었',
 '다',
 '.',
 '이런',
 '이유',
 '로',
 '초기',
 '에',
 '는',
 '도움말',
 '을',
 '옮기',
 '거나',
 '쓰',
 '는',
 '일',
 '에',
 '어려움',
 '을',
 '겪',
 '었',
 '다',
 '.',
 '이런',
 '어려움',
 '이',
 '있',
 '었',
 '는데',
 '도',
 '위',
 '키',
 '백과',
 '통계',
 '로',
 '는',
 ',',
 '2002',
 '년',
 '10',
 '월',
 '에서',
 '2003',
 '년',
 '7',
 '월',
 '까지',
 '열',
 'ㄹ',
 '달',
 '사이',
 '에',
 '글',
 '이',
 '13',
 '개',
 '에서',
 '159',
 '개',
 '로

In [114]:
mecab_counter = collections.defaultdict(int)
# mecab tokenizer 개수 확인 (시간 오래 걸림)
with zipfile.ZipFile(os.path.join(kowiki_dir, 'kowiki.txt.zip')) as z:
    with z.open('kowiki.txt') as f:
        for i, line in enumerate(f):
            if i >= 100000:
                break
            line = line.decode('utf-8').strip()
            for w in mecab.parse(line):
                mecab_counter[w] += 1
len(mecab_counter)

6974

In [115]:
list(mecab_counter.items())[:100]

[('지', 430927),
 ('미', 55246),
 ('\t', 6944728),
 ('I', 436638),
 ('C', 1566487),
 (',', 48788999),
 ('*', 34821900),
 ('F', 3640116),
 ('\n', 7044728),
 ('카', 29247),
 ('터', 40068),
 ('N', 6772807),
 ('P', 996326),
 ('인', 352118),
 ('명', 446921),
 ('E', 2021350),
 ('O', 420452),
 ('S', 2114211),
 ('제', 117558),
 ('임', 27107),
 ('스', 143293),
 ('얼', 2961),
 ('T', 3580629),
 ('"', 21543),
 ('Y', 110622),
 ('G', 2777857),
 ('주', 146650),
 ('니', 42630),
 ('어', 193507),
 ('(', 69871),
 ('1', 115707),
 ('9', 55493),
 ('2', 62377),
 ('4', 31403),
 ('년', 139903),
 ('B', 656781),
 ('0', 72055),
 ('월', 56101),
 ('일', 135125),
 ('~', 82635),
 (')', 69954),
 ('는', 448208),
 ('J', 1372630),
 ('X', 1194095),
 ('민', 69113),
 ('당', 62540),
 ('o', 567747),
 ('m', 288457),
 ('p', 278238),
 ('u', 285841),
 ('n', 716093),
 ('d', 279875),
 ('/', 2815913),
 ('+', 1129629),
 ('출', 23865),
 ('신', 69880),
 ('국', 157332),
 ('3', 36097),
 ('대', 236427),
 ('통', 55657),
 ('령', 21439),
 ('7', 28258),
 ('8', 33276)

# 한국어 위키 다운로드 및 전처리

In [83]:
# 한국어 위키 최신 dump 버전 다운로드
!wget https://dumps.wikimedia.org/kowiki/latest/kowiki-latest-pages-meta-current.xml.bz2

--2021-01-26 05:10:27--  https://dumps.wikimedia.org/kowiki/latest/kowiki-latest-pages-meta-current.xml.bz2
Resolving dumps.wikimedia.org (dumps.wikimedia.org)... 208.80.155.106, 2620:0:861:4:208:80:155:106
Connecting to dumps.wikimedia.org (dumps.wikimedia.org)|208.80.155.106|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 918835390 (876M) [application/octet-stream]
Saving to: ‘kowiki-latest-pages-meta-current.xml.bz2’

kowiki-latest-pages 100%[===================>] 876.27M  4.98MB/s    in 2m 56s  

2021-01-26 05:13:24 (4.97 MB/s) - ‘kowiki-latest-pages-meta-current.xml.bz2’ saved [918835390/918835390]



In [84]:
# WikiExtractor 다운로드
!wget https://github.com/paul-hyun/web-crawler/raw/master/WikiExtractor.py

--2021-01-26 05:13:31--  https://github.com/paul-hyun/web-crawler/raw/master/WikiExtractor.py
Resolving github.com (github.com)... 140.82.112.4
Connecting to github.com (github.com)|140.82.112.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/paul-hyun/web-crawler/master/WikiExtractor.py [following]
--2021-01-26 05:13:31--  https://raw.githubusercontent.com/paul-hyun/web-crawler/master/WikiExtractor.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 119222 (116K) [text/plain]
Saving to: ‘WikiExtractor.py’

WikiExtractor.py    100%[===================>] 116.43K  --.-KB/s    in 0.02s   

2021-01-26 05:13:31 (4.66 MB/s) - ‘WikiExtractor.py’ saved [119222/119222]



In [85]:
# 현재 폴더 파일 목록 확인
os.listdir('.')

['.config',
 'drive',
 'kowiki.txt.zip',
 'kowiki.txt',
 'WikiExtractor.py',
 'kowiki-latest-pages-meta-current.xml.bz2',
 'ko_32000.model',
 'ko_32000.vocab',
 'sample_data']

In [86]:
# WikiExtractor 실행 (30분 이상 오랜 시간 소요 됨)
# -o: 출력할 폴더
# --json: json format으로 출력
os.system(f"python WikiExtractor.py -o kowiki --json kowiki-latest-pages-meta-current.xml.bz2")

0

In [87]:
with open(os.path.join('kowiki', 'AA', 'wiki_00')) as f:
    for i, line in enumerate(f):
        if i >= 10:
            break
        line = line.strip() 
        print(line)

{"id": "5", "url": "https://ko.wikipedia.org/wiki?curid=5", "title": "지미 카터", "text": "지미 카터\n\n제임스 얼 \"지미\" 카터 주니어(, 1924년 10월 1일 ~ )는 민주당 출신 미국 39대 대통령 (1977년 ~ 1981년)이다.\n\n지미 카터는 조지아주 섬터 카운티 플레인스 마을에서 태어났다. 조지아 공과대학교를 졸업하였다. 그 후 해군에 들어가 전함·원자력·잠수함의 승무원으로 일하였다. 1953년 미국 해군 대위로 예편하였고 이후 땅콩·면화 등을 가꿔 많은 돈을 벌었다. 그의 별명이 \"땅콩 농부\" (Peanut Farmer)로 알려졌다.\n\n1962년 조지아 주 상원 의원 선거에서 낙선하나 그 선거가 부정선거 였음을 입증하게 되어 당선되고, 1966년 조지아 주지사 선거에 낙선하지만, 1970년 조지아 주지사를 역임했다. 대통령이 되기 전 조지아주 상원의원을 두번 연임했으며, 1971년부터 1975년까지 조지아 지사로 근무했다. 조지아 주지사로 지내면서, 미국에 사는 흑인 등용법을 내세웠다.\n\n1976년 미합중국 (미국) 제39대 대통령 선거에 민주당 후보로 출마하여 도덕주의 정책으로 내세워서, 많은 지지를 받고 제럴드 포드 (당시 미국 대통령) 를 누르고 당선되었다.\n\n카터 대통령은 에너지 개발을 촉구했으나 공화당의 반대로 무산되었다.\n\n카터는 이집트와 이스라엘을 조정하여, 캠프 데이비드에서 안와르 사다트 대통령과 메나헴 베긴 수상과 함께 중동 평화를 위한 캠프데이비드 협정을 체결했다.\n\n그러나 이것은 공화당과 미국의 유대인 단체의 반발을 일으켰다. 1979년 백악관에서 양국 간의 평화조약으로 이끌어졌다. 또한 소련과 제2차 전략 무기 제한 협상에 조인했다.\n\n카터는 1970년대 후반 당시 대한민국 등 인권 후진국의 국민들의 인권을 지키기 위해 노력했으며, 취임 이후 계속해서 도덕정치를 내세웠다.\n\n그러나 주 이란 미국 대사관 인질 사건에서 인질 구

In [116]:
def list_wiki(dirname):
    """
    위키 목록을 읽어들임
    :param dirname: 위키 dir
    :return: 위키 파일 목록
    """
    filepaths = []
    filenames = os.listdir(dirname)
    for filename in filenames:
        filepath = os.path.join(dirname, filename)

        if os.path.isdir(filepath):
            filepaths.extend(list_wiki(filepath))
        else:
            find = re.findall(r"wiki_[0-9][0-9]", filepath)
            if 0 < len(find):
                filepaths.append(filepath)
    return sorted(filepaths)

In [117]:
filepaths = []
dirnames = os.listdir('kowiki')
for dirname in dirnames:
    dirpath = os.path.join('kowiki', dirname)
    filenames = os.listdir(dirpath)
    for filename in filenames:
        if filename.startswith('wiki_'):
            filepath = os.path.join(dirpath, filename)
            filepaths.append(filepath)
filepaths = sorted(filepaths)
print(len(filepaths), filepaths[:10])

738 ['kowiki/AA/wiki_00', 'kowiki/AA/wiki_01', 'kowiki/AA/wiki_02', 'kowiki/AA/wiki_03', 'kowiki/AA/wiki_04', 'kowiki/AA/wiki_05', 'kowiki/AA/wiki_06', 'kowiki/AA/wiki_07', 'kowiki/AA/wiki_08', 'kowiki/AA/wiki_09']


In [118]:
def trim_text(item):
    """
    한 위키 문서 내의 여러줄띄기(\n\n...)를 한줄띄기로(\n)로 변경
    :param item: 위키 항목
    :return: text의 여러줄 new line을 한 줄 new line으로 변경한 json data
    """
    data = json.loads(item)
    text = data["text"]
    value = list(filter(lambda x: len(x) > 0, text.split('\n')))
    data["text"] = "\n".join(value)
    return data

In [119]:
# 여러줄띄기(\n\n...)를 한줄띄기로(\n)로 변경
dataset = []
for filepath in tqdm(filepaths):
    with open(filepath, "r") as f:
        for line in f:
            line = line.strip()
            if line:
                dataset.append(trim_text(line))
print(len(dataset))


531653


In [120]:
# 위키를 한 파일로 저장
with open("kowiki.txt", "w") as f:
    for data in tqdm(dataset):
        f.write(data["text"])
        f.write("\n\n\n\n")

In [121]:
# 파일 내용 확인
with open("kowiki.txt") as f:
    for i, line in enumerate(f):
        if i >= 30:
            break
        line = line.strip()
        print(line)

지미 카터
제임스 얼 "지미" 카터 주니어(, 1924년 10월 1일 ~ )는 민주당 출신 미국 39대 대통령 (1977년 ~ 1981년)이다.
지미 카터는 조지아주 섬터 카운티 플레인스 마을에서 태어났다. 조지아 공과대학교를 졸업하였다. 그 후 해군에 들어가 전함·원자력·잠수함의 승무원으로 일하였다. 1953년 미국 해군 대위로 예편하였고 이후 땅콩·면화 등을 가꿔 많은 돈을 벌었다. 그의 별명이 "땅콩 농부" (Peanut Farmer)로 알려졌다.
1962년 조지아 주 상원 의원 선거에서 낙선하나 그 선거가 부정선거 였음을 입증하게 되어 당선되고, 1966년 조지아 주지사 선거에 낙선하지만, 1970년 조지아 주지사를 역임했다. 대통령이 되기 전 조지아주 상원의원을 두번 연임했으며, 1971년부터 1975년까지 조지아 지사로 근무했다. 조지아 주지사로 지내면서, 미국에 사는 흑인 등용법을 내세웠다.
1976년 미합중국 (미국) 제39대 대통령 선거에 민주당 후보로 출마하여 도덕주의 정책으로 내세워서, 많은 지지를 받고 제럴드 포드 (당시 미국 대통령) 를 누르고 당선되었다.
카터 대통령은 에너지 개발을 촉구했으나 공화당의 반대로 무산되었다.
카터는 이집트와 이스라엘을 조정하여, 캠프 데이비드에서 안와르 사다트 대통령과 메나헴 베긴 수상과 함께 중동 평화를 위한 캠프데이비드 협정을 체결했다.
그러나 이것은 공화당과 미국의 유대인 단체의 반발을 일으켰다. 1979년 백악관에서 양국 간의 평화조약으로 이끌어졌다. 또한 소련과 제2차 전략 무기 제한 협상에 조인했다.
카터는 1970년대 후반 당시 대한민국 등 인권 후진국의 국민들의 인권을 지키기 위해 노력했으며, 취임 이후 계속해서 도덕정치를 내세웠다.
그러나 주 이란 미국 대사관 인질 사건에서 인질 구출 실패를 이유로 1980년 대통령 선거에서 공화당의 로널드 레이건 후보에게 져 결국 재선에 실패했다. 또한 임기 말기에 터진 소련의 아프가니스탄 침공 사건으로 인해 1980년 하계 올림픽에 반공국가들의 보이콧을

In [122]:
# 압축
!zip kowiki.txt.zip kowiki.txt

updating: kowiki.txt (deflated 64%)


In [123]:
# 압축파일 보관
shutil.move('kowiki.txt.zip', os.path.join(kowiki_dir, 'kowiki.txt.zip'))
os.listdir(kowiki_dir)

['kowiki.txt.zip']